In [28]:
from sklearn import svm
from gameClasses import *
from sklearn.linear_model import LogisticRegression
# from plotConMatrix import plot_confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.neural_network import MLPRegressor


import math as m
print("Importing Classes")

Importing Classes


In [3]:
def movesRandom(state,player):
    board = [state,True]
    while not(state.isTerminal()) and board[1]:
        print(state,board[1])
        moves = state.legalMoves(player)
        move = random.choice(moves)
        board = state.result(move)
        
        if state.isTerminal() or len(moves) == 1:
            break
    # print(board[0].board)
    return board

# Data Generation

In [97]:
features = []
labels = []



for i in range(0,3):
    initial = [0,0,np.full((2,6), 4)]
    
    initial[2][1][random.randint(0,5)] += 1
    initial[2][0][random.randint(0,5)] -= 1
    initial[2][1][random.randint(0,5)] += 1
    initial[2][0][random.randint(0,5)] -= 1
    initial[2][1][random.randint(0,5)] += 1
    initial[2][0][random.randint(0,5)] -= 1
    
    print(initial[2])
    a = MancalaState(initial)
    
    aiDepth = random.randint(4,5)
    plDepth = random.randint(1,2)
    
    count = 0
    count1 = 0 
    while not(a.isTerminal()):
        count += 1
        a = minimax(a,aiDepth, 0)[0]

        if count > 4:
    #         print(0,a.board.flatten())
            count1 += 1
            features += [list(a.board.flatten())]  
        if a.isTerminal():
            break
        #pl1
        a = minimax(a,plDepth, 1)[0]
        if count > 4:
    #         print(1,a.board.flatten())
            count1 += 1
            features += [list(a.board.flatten())]
    labels += [a.aiScore-a.plScore] * count1
    print(a.aiScore-a.plScore)
    del a


[[3 3 4 3 4 4]
 [5 4 5 4 4 5]]
14
[[4 3 4 3 4 3]
 [5 6 4 4 4 4]]
32
[[4 4 3 3 3 4]
 [6 4 5 4 4 4]]
20


In [92]:
features = np.array(features)
labels = np.array(labels)
# np.save("features", features)
# np.save("labels", labels)
print(features,labels)

[list([5, 0, 3, 5, 1, 0, 0, 3, 0, 2, 0, 10]) 6 13
 list([5, 0, 3, 5, 0, 0, 0, 0, 1, 3, 0, 10])
 list([6, 1, 0, 5, 0, 0, 0, 0, 1, 3, 0, 10]) 7 15
 list([6, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 11])
 list([7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 11]) 7 22
 list([7, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 12])
 list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) 8 40
 list([5, 10, 0, 9, 2, 8, 0, 0, 0, 0, 0, 1]) 4 9
 list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0])
 list([3, 1, 0, 1, 8, 0, 1, 1, 2, 1, 9, 0]) 6 15
 list([4, 2, 1, 2, 0, 1, 1, 1, 0, 2, 0, 0])
 list([5, 0, 1, 2, 0, 1, 1, 1, 0, 2, 0, 0]) 8 27
 list([5, 0, 0, 2, 0, 1, 1, 0, 0, 2, 0, 0])
 list([0, 0, 0, 2, 0, 1, 2, 1, 1, 3, 0, 0]) 9 29
 list([0, 0, 0, 0, 0, 1, 2, 1, 0, 0, 1, 1])
 list([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]) 11 37] [-32 -32 -32 -32 -32 -32 -32  28  28 -26 -26 -26 -26 -26 -26 -26]


In [99]:
features = np.load("features.npy")
labels = np.load("labels.npy")
def learnLogisticalReg(x_train, y_train):
    model = LogisticRegression(solver = 'lbfgs',  multi_class ='multinomial')
    model.fit(x_train, y_train)
    return model

In [100]:
def evaluate(X_test,y_test,model,title1):
    y_pred = model.predict(X_test)
    np.set_printoptions(precision=2)
    print(np.sum((y_test-y_pred)**2)/len(X_test))
    print(title1,"Accuracy", accuracy_score(y_test, y_pred))

In [101]:
size = len(features)
T_size = m.floor(size*.70)
model3 = learnLogisticalReg(features[0:T_size],labels[0:T_size])
T_size

/Users/mazeyarmoeini/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:947: ConvergenceWarning: lbfgs failed to converge. Increase the number of iterations.
  "of iterations.", ConvergenceWarning)


922

In [102]:
evaluate(features[size-T_size:],labels[size-T_size:],model3,"Logistical Regression")

115.82212581344902
Logistical Regression Accuracy 0.38611713665943603


In [103]:
def MLP_Regressor(x_train, y_train):
    model = MLPRegressor(random_state=0, activation='relu', solver='lbfgs', alpha=1e-5, hidden_layer_sizes=(16, 12, 8, 4))
    model.fit(x_train, y_train)
    return model

In [104]:
def evaluate_MLP(X_test,y_test,model,title1):
    y_pred = model.predict(X_test)
    print(list(zip(y_test,y_pred)))
    print(title1,"Accuracy", np.sum((y_test-y_pred)**2)/len(y_test))

In [105]:
size = len(features)
T_size = m.floor(size*.70)
model4 = MLP_Regressor(features[0:T_size],labels[0:T_size])
T_size

922

In [106]:
evaluate_MLP(features[size-T_size:],labels[size-T_size:],model4,"MLP Regression")

[(-20, -21.2495161516767), (-20, -23.026247984055434), (-20, -22.5489109323518), (-20, -22.44043194513998), (-20, -20.46579079214717), (-20, -25.829855448435353), (-20, -9.767995743758881), (-18, -23.94213938617589), (-18, -18.626137242934014), (-18, -18.62240766709716), (-18, -18.296793951614813), (-18, -17.502153740990064), (-18, -19.014821573899482), (-18, -19.83064724100847), (-18, -18.346785729496464), (-18, -20.832181100518476), (-18, -16.99317375983938), (-18, -14.187068424444448), (-18, -19.886369653641793), (-18, -23.630728690092063), (-18, -24.08459471768205), (-18, -25.246778536332975), (-18, -25.148730044935967), (-18, -25.013586119343394), (-18, -24.409326394683873), (-18, -9.767995743758881), (-16, -15.329772032843163), (-16, -17.013937975465613), (-16, -19.0784884268012), (-16, -18.31001377244507), (-16, -18.38991638699119), (-16, -18.297127644984766), (-16, -9.767995743758881), (-34, -29.560090644374412), (-34, -33.762022225944335), (-34, -34.73054020785156), (-34, -27.